<a href="https://colab.research.google.com/github/amoyag/Biofisica/blob/main/S4_lambdaswitch_profesor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Banco de módulos

In [ ]:
#@title Módulo 0. Librerías
# -*- coding: utf-8 -*-
# =========================
# Módulo 0: Librerías y estilo
# =========================
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from IPython.display import display

plt.style.use('classic')
sns.set_theme(style="whitegrid")

# Paleta consistente
PALETTE = {
    "CI": "#1f77b4",   # azul
    "Cro": "#d62728",  # rojo
    "nullcline_CI": "#1f77b4",
    "nullcline_Cro": "#d62728",
    "vector_field": "#7f7f7f"
}


In [ ]:
#@title Módulo 1. Modelo Lambda Switch
# =========================
# Módulo 1: Modelo del Lambda Switch
# =========================
def hill_repression(x, K, n):
    """
    Función de represión tipo Hill: g(x) = 1 / (1 + (x/K)^n)
    x: concentración del represor
    K: constante de semisaturación
    n: coeficiente de Hill (cooperatividad)
    """
    return 1.0 / (1.0 + (x / K)**n)

def model_lambda_switch(t, y, params):
    """
    Modelo mínimo del interruptor lambda con represión mutua y expresión basal.

    Variables:
      y = [CI, Cro]

    Ecuaciones:
      dCI/dt  = beta_CI * g_Cro(Cro) + beta0_CI - alpha_CI * CI
      dCro/dt = beta_Cro * g_CI(CI)  + beta0_Cro - alpha_Cro * Cro

    Donde g_X(.) es la función de represión Hill del represor correspondiente.

    params esperados:
      params["CI"]  = {"beta": ..., "beta0": ..., "alpha": ..., "K": ..., "n": ...}
      params["Cro"] = {"beta": ..., "beta0": ..., "alpha": ..., "K": ..., "n": ...}
      (opcional) params["input"] = función u(t) para modular producción (ver abajo)
      (opcional) params["input_effect"] = dict con claves "CI" y/o "Cro" y ganancia k

    Nota: Si se desea entrada externa, se implementa como modulación multiplicativa
          de la producción máxima: beta_eff = beta * (1 + k * u(t)).
    """
    CI, Cro = y

    # Parám. CI
    beta_CI  = params["CI"]["beta"]
    beta0_CI = params["CI"]["beta0"]
    alpha_CI = params["CI"]["alpha"]
    K_CI     = params["CI"]["K"]
    n_CI     = params["CI"].get("n", 2)

    # Parám. Cro
    beta_Cro  = params["Cro"]["beta"]
    beta0_Cro = params["Cro"]["beta0"]
    alpha_Cro = params["Cro"]["alpha"]
    K_Cro     = params["Cro"]["K"]
    n_Cro     = params["Cro"].get("n", 2)

    # Entrada externa opcional (p. ej., señal de daño SOS)
    u = 0.0
    if "input" in params and callable(params["input"]):
        u = params["input"](t)  # Sintaxis correcta: función de entrada evaluada en t

    # Modulación opcional de betas por entrada externa
    if "input_effect" in params:
        k_CI  = params["input_effect"].get("CI", 0.0)
        k_Cro = params["input_effect"].get("Cro", 0.0)
        beta_CI_eff  = beta_CI  * (1.0 + k_CI  * u)
        beta_Cro_eff = beta_Cro * (1.0 + k_Cro * u)
    else:
        beta_CI_eff, beta_Cro_eff = beta_CI, beta_Cro

    # Represiones Hill
    g_Cro_on_CI  = hill_repression(Cro, K_Cro, n_Cro)  # Cro reprime CI
    g_CI_on_Cro  = hill_repression(CI,  K_CI,  n_CI)   # CI  reprime Cro

    # Ecuaciones dinámicas
    dCI_dt  = beta_CI_eff  * g_Cro_on_CI + beta0_CI  - alpha_CI  * CI
    dCro_dt = beta_Cro_eff * g_CI_on_Cro + beta0_Cro - alpha_Cro * Cro

    return [dCI_dt, dCro_dt]


In [ ]:
#@title Módulo 2: Construcción de configuración
# =========================
# Módulo 2: Configuración
# =========================
def build_config(input_type="none", t_span=(0.0, 200.0), y0=None):
    """
    Crea la configuración por defecto del interruptor lambda.

    input_type: "none" | "step" | "pulse"
      - "none": sin entrada externa.
      - "step": u(t)=1 para t>=t_on.
      - "pulse": u(t)=1 en [t_on, t_on+dur].

    t_span: tupla (t0, tf)
    y0: condiciones iniciales [CI0, Cro0]; si es None, usa [0.1, 0.1]
    """
    # Entrada externa (opcional)
    if input_type == "none":
        input_func = lambda t: 0.0
    elif input_type == "step":
        t_on = 20.0
        input_func = lambda t, t_on=t_on: 1.0 if t >= t_on else 0.0
    elif input_type == "pulse":
        t_on, dur = 20.0, 30.0
        input_func = lambda t, t_on=t_on, dur=dur: 1.0 if (t >= t_on and t <= t_on + dur) else 0.0
    else:
        raise ValueError("input_type debe ser 'none', 'step' o 'pulse'.")

    # Parámetros base (ajustables en los ejercicios)
    params = {
        "input": input_func,
        # Ganancias de entrada externa (por defecto 0: sin efecto)
        "input_effect": {"CI": 0.0, "Cro": 0.0},
        "CI":  {"beta": 2.0, "beta0": 0.05, "alpha": 0.2, "K": 1.0, "n": 2},
        "Cro": {"beta": 2.0, "beta0": 0.05, "alpha": 0.2, "K": 1.0, "n": 2},
    }

    if y0 is None:
        y0 = [0.1, 0.1]

    return params, t_span, y0

In [ ]:
#@title Módulo 3. Simulación

# =========================
# Módulo 3: Simulación
# =========================
def run_simulation(model_func, params, t_span, y0, t_eval=None, rtol=1e-6, atol=1e-9):
    """
    Ejecuta la simulación con solve_ivp y devuelve tiempos y trayectorias.
    """
    if t_eval is None:
        t_eval = np.linspace(t_span[0], t_span[1], 2000)

    sol = solve_ivp(
        model_func, t_span, y0,
        args=(params,),
        t_eval=t_eval, rtol=rtol, atol=atol, method="RK45"
    )
    return sol.t, sol.y


In [ ]:
#@title Módulo 4. Visualización

# =========================
# Módulo 4: Visualización (series temporales, nulclinas, campo de fases)
# =========================
def plot_timeseries(t, y, title="Dinámica temporal del interruptor lambda"):
    """
    Dibuja las series temporales de CI y Cro.
    """
    CI, Cro = y[0], y[1]
    plt.figure(figsize=(8, 5))
    plt.plot(t, CI,  color=PALETTE["CI"],  label="CI")
    plt.plot(t, Cro, color=PALETTE["Cro"], label="Cro")
    plt.xlabel("Tiempo")
    plt.ylabel("Concentración")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

def compute_nullclines(params, ci_range, cro_range):
    """
    Calcula nulclinas analíticas aproximadas:
      - Nulclina CI: dCI/dt=0 => CI = (beta_CI_eff*g_Cro + beta0_CI)/alpha_CI
      - Nulclina Cro: Cro = (beta_Cro_eff*g_CI + beta0_Cro)/alpha_Cro

    Devuelve arrays (CI_nc, Cro_nc) discretizados sobre las rejillas cro_range y ci_range respectivamente.
    """
    # Extrae parámetros
    beta_CI  = params["CI"]["beta"]
    beta0_CI = params["CI"]["beta0"]
    alpha_CI = params["CI"]["alpha"]
    K_Cro    = params["Cro"]["K"]
    n_Cro    = params["Cro"].get("n", 2)

    beta_Cro  = params["Cro"]["beta"]
    beta0_Cro = params["Cro"]["beta0"]
    alpha_Cro = params["Cro"]["alpha"]
    K_CI      = params["CI"]["K"]
    n_CI      = params["CI"].get("n", 2)

    # Suponemos sin modulación externa para las nulclinas (u(t) ~ 0)
    beta_CI_eff, beta_Cro_eff = beta_CI, beta_Cro

    # Nulclina de CI en función de Cro
    g_Cro = 1.0 / (1.0 + (cro_range / K_Cro)**n_Cro)
    CI_nc = (beta_CI_eff * g_Cro + beta0_CI) / alpha_CI

    # Nulclina de Cro en función de CI
    g_CI = 1.0 / (1.0 + (ci_range / K_CI)**n_CI)
    Cro_nc = (beta_Cro_eff * g_CI + beta0_Cro) / alpha_Cro

    return CI_nc, Cro_nc

def plot_phase_portrait(model_func, params, bounds=(0, 3, 0, 3), grid=25, t_arrows=0.0):
    """
    Dibuja el campo de fases (vector field) y las nulclinas sobre el plano (CI, Cro).
    bounds: (CI_min, CI_max, Cro_min, Cro_max)
    grid: densidad de rejilla para el campo de fases
    t_arrows: tiempo al que evaluar la entrada externa si se desea (0 por defecto)
    """
    CI_min, CI_max, Cro_min, Cro_max = bounds
    ci = np.linspace(CI_min, CI_max, grid)
    cro = np.linspace(Cro_min, Cro_max, grid)
    CI_mesh, Cro_mesh = np.meshgrid(ci, cro)

    # Campo de fases
    dCI = np.zeros_like(CI_mesh)
    dCro = np.zeros_like(Cro_mesh)
    for i in range(grid):
        for j in range(grid):
            dydt = model_func(t_arrows, [CI_mesh[i, j], Cro_mesh[i, j]], params)
            dCI[i, j], dCro[i, j] = dydt

    # Nulclinas
    CI_nc, Cro_nc = compute_nullclines(params, ci_range=ci, cro_range=cro)

    plt.figure(figsize=(7, 6))
    # Vector field
    plt.quiver(CI_mesh, Cro_mesh, dCI, dCro, color=PALETTE["vector_field"], angles='xy', scale_units='xy', scale=1.5, alpha=0.6, width=0.003)
    # Nulclina CI: CI vs Cro
    plt.plot(CI_nc, cro, color=PALETTE["nullcline_CI"],  lw=2, label="dCI/dt = 0")
    # Nulclina Cro: CI vs Cro
    plt.plot(ci, Cro_nc, color=PALETTE["nullcline_Cro"], lw=2, label="dCro/dt = 0")

    plt.xlabel("CI")
    plt.ylabel("Cro")
    plt.xlim(CI_min, CI_max)
    plt.ylim(Cro_min, Cro_max)
    plt.title("Campo de fases y nulclinas (interruptor lambda)")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
#@title Módulo 5. Análisis

# =========================
# Módulo 5: Análisis (estado final, cuencas de atracción)
# =========================
def classify_state(CI_final, Cro_final, ratio_thresh=1.0):
    """
    Clasifica el estado final:
      - 'lysogeny' si CI_final / Cro_final >= ratio_thresh
      - 'lysis'     si CI_final / Cro_final <  ratio_thresh

    ratio_thresh=1.0 implica comparación directa CI vs Cro.
    """
    eps = 1e-9
    ratio = (CI_final + eps) / (Cro_final + eps)
    return "lisogenia" if ratio >= ratio_thresh else "lisis"

def compute_outcome(t, y, settle_window=0.1):
    """
    Devuelve estado final y valores finales promediados al término de la simulación.
    settle_window: fracción final del intervalo temporal para promediar (por defecto 10%).
    """
    N = max(5, int(len(t) * settle_window))
    CI_final = float(np.mean(y[0][-N:]))
    Cro_final = float(np.mean(y[1][-N:]))
    state = classify_state(CI_final, Cro_final)
    return state, CI_final, Cro_final

def basin_of_attraction(model_func, params, t_span, ci0_range, cro0_range, steps=25, t_eval=None):
    """
    Explora cuencas de atracción: para una rejilla de condiciones iniciales (CI0, Cro0),
    simula y clasifica el estado final. Devuelve:
      - grid_CI0, grid_Cro0 (mallas)
      - outcome_matrix (0=lisis, 1=lisogenia)
    """
    ci0_vals  = np.linspace(ci0_range[0],  ci0_range[1],  steps)
    cro0_vals = np.linspace(cro0_range[0], cro0_range[1], steps)
    grid_CI0, grid_Cro0 = np.meshgrid(ci0_vals, cro0_vals)

    outcome_matrix = np.zeros_like(grid_CI0, dtype=int)

    for i in range(steps):
        for j in range(steps):
            y0 = [grid_CI0[i, j], grid_Cro0[i, j]]
            t, y = run_simulation(model_func, params, t_span, y0, t_eval=t_eval)
            state, _, _ = compute_outcome(t, y)
            outcome_matrix[i, j] = 1 if state == "lisogenia" else 0

    return grid_CI0, grid_Cro0, outcome_matrix

def plot_basin(grid_CI0, grid_Cro0, outcome_matrix, cmap="RdBu", title="Cuencas de atracción"):
    """
    Dibuja el mapa de cuencas de atracción:
      rojo -> lisis (0)
      azul -> lisogenia (1)
    """
    plt.figure(figsize=(7, 6))
    im = plt.imshow(
        outcome_matrix, origin="lower", aspect="auto",
        extent=[grid_CI0.min(), grid_CI0.max(), grid_Cro0.min(), grid_Cro0.max()],
        cmap=cmap, vmin=0, vmax=1
    )
    plt.colorbar(im, label="Estado final (0=lisis, 1=lisogenia)")
    plt.xlabel("CI₀ (condición inicial)")
    plt.ylabel("Cro₀ (condición inicial)")
    plt.title(title)
    plt.tight_layout()
    plt.show()


In [ ]:
#@title Módulo 6. Utilidades

# =========================
# Módulo 6: Utilidades (entrada, helpers)
# =========================
def make_step(t_on=20.0, amplitude=1.0):
    """
    Genera una entrada escalón: u(t) = amplitude para t >= t_on; 0 en otro caso.
    """
    return lambda t, t_on=t_on, amplitude=amplitude: amplitude if t >= t_on else 0.0

def make_pulse(t_on=20.0, duration=30.0, amplitude=1.0):
    """
    Genera una entrada pulso: u(t) = amplitude en [t_on, t_on+duration]; 0 en otro caso.
    """
    return lambda t, t_on=t_on, duration=duration, amplitude=amplitude: (
        amplitude if (t >= t_on and t <= t_on + duration) else 0.0
    )

def copy_params(params):
    """
    Copia segura del diccionario de parámetros (copia superficial por claves de primer nivel
    y copia por clave para subdiccionarios CI y Cro).
    """
    new_params = {}
    for k, v in params.items():
        if isinstance(v, dict):
            new_params[k] = {kk: vv for kk, vv in v.items()}
        else:
            new_params[k] = v
    return new_params


### Ejercicio 1. Visualización de la biestabilidad y dinámica temporal**

**Objetivo:**  
Explorar cómo el interruptor del fago lambda presenta dos estados estables (lisogenia y lisis) y cómo la dinámica temporal depende de los parámetros del sistema.




**Instrucciones:**

1.  Ajusta los parámetros
    *   **$\beta_{CI}$** y **$\beta_{Cro}$**: tasas máximas de producción de CI y Cro.
    *   **$\alpha_{CI}$** y **$\alpha_{Cro}$**: tasas de degradación.
    *   **$\beta_{oCI}$** y **$\beta_{oCro}$**: expresión basal.
2.  Observa:
    *   Las curvas temporales de CI y Cro.
    *   El **campo de fases** con las nulclinas y el vector field.
3.  Analiza:
    *   ¿Qué ocurre si aumentas la tasa de producción de CI?
    *   ¿Cómo cambia la posición de los atractores?
    *   ¿Por qué el sistema se estabiliza en uno de dos estados y no en un punto intermedio?

El campo de fases es una herramienta para visualizar cómo evoluciona el sistema en el plano formado por las dos variables: CI y Cro. Cada punto representa un estado posible (una combinación de concentraciones), y las flechas indican la dirección y velocidad del cambio en ese punto.
Las nulclinas son curvas donde la derivada de una variable es cero:

- $\frac{dCI}{dt} = 0$ (línea azul): puntos donde CI no cambia.
- $\frac{dCro}{dt} = 0$ (línea roja): puntos donde Cro no cambia.

Los puntos donde ambas nulclinas se cruzan son puntos fijos:

Si los vectores alrededor apuntan hacia el cruce, es estable (atractor).
Si se alejan, es inestable (repulsor).

El campo de fases muestra la estructura global del sistema: hacia dónde se mueve desde cada región y qué atractores existen.

**Preguntas para reflexionar:**

*   ¿Qué parámetros favorecen la lisogenia? ¿Cuáles favorecen la lisis?
*   ¿Por qué la biestabilidad es importante para la toma de decisiones biológicas?

***




In [ ]:

# =========================
# Ejercicio 1: Bistabilidad y dinámica temporal (Interactivo)
# =========================
import ipywidgets as widgets
from IPython.display import display

# Sliders para parámetros clave
beta_CI_slider  = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='β_CI')
beta_Cro_slider = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='β_Cro')
alpha_CI_slider = widgets.FloatSlider(value=0.2, min=0.05, max=0.5, step=0.01, description='α_CI')
alpha_Cro_slider= widgets.FloatSlider(value=0.2, min=0.05, max=0.5, step=0.01, description='α_Cro')
beta0_CI_slider = widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.01, description='β₀_CI')
beta0_Cro_slider= widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.01, description='β₀_Cro')

# Función interactiva
def simulate_bistability(beta_CI, beta_Cro, alpha_CI, alpha_Cro, beta0_CI, beta0_Cro):
    # Configuración base
    #aqui podemos definir condiciones iniciales de Ci y Cro, para que el sistema vaya a un atractor o a otro
    # y0=[0.2, 4.0] empezamos cerca del atractor superior (lisis)
    # y0=[4.0, 0.2] empezamos cerca del atractor inferior
    params, t_span, y0 = build_config(input_type="none", t_span=(0, 200), y0=[0.1, 0.5])

    # Ajustar parámetros según sliders
    params["CI"]["beta"]  = beta_CI
    params["CI"]["alpha"] = alpha_CI
    params["CI"]["beta0"] = beta0_CI
    params["Cro"]["beta"]  = beta_Cro
    params["Cro"]["alpha"] = alpha_Cro
    params["Cro"]["beta0"] = beta0_Cro

    # Simulación
    t, y = run_simulation(model_lambda_switch, params, t_span, y0)

    # Visualización: series temporales
    plot_timeseries(t, y, title="Dinámica temporal: CI vs Cro")

    # Visualización: campo de fases y nulclinas
    plot_phase_portrait(model_lambda_switch, params, bounds=(0, 20, 0, 20), grid=25)

    # Estado final
    state, CI_final, Cro_final = compute_outcome(t, y)
    print(f"Estado final: {state}")
    print(f"CI final ≈ {CI_final:.2f}, Cro final ≈ {Cro_final:.2f}")

# Crear interfaz interactiva
widgets.interactive(simulate_bistability,
    beta_CI=beta_CI_slider,
    beta_Cro=beta_Cro_slider,
    alpha_CI=alpha_CI_slider,
    alpha_Cro=alpha_Cro_slider,
    beta0_CI=beta0_CI_slider,
    beta0_Cro=beta0_Cro_slider
)

#### Cómo se ve la bistabilidad clásica

En el interruptor lambda típico, con parámetros que favorecen la represión mutua y la cooperatividad (por ejemplo, β altos y α bajos), el sistema presenta:

Dos puntos fijos estables:

Uno con CI alto y Cro bajo (lisogenia).
Otro con Cro alto y CI bajo (lisis).


Un punto fijo inestable en el centro:
Actúa como frontera entre las dos cuencas de atracción. Si el sistema empieza cerca de este punto, pequeñas diferencias deciden el destino.

En el campo de fases:

Las flechas divergen del punto central y convergen hacia los extremos.
Las nulclinas se cruzan tres veces, pero solo dos cruces son atractores estables.

#### multistabilidad con parámetros actuales
Con los parámetros que has usado:

β_CI = 1.50
β_Cro = 1.50
α_CI = 0.30
α_Cro = 0.37
β₀_CI = 0.00
β₀_Cro = 0.00

El sistema muestra tres puntos fijos estables:

Arriba a la izquierda: CI ≈ 0.4, Cro ≈ 3.5 → estado tipo lisis.
Centro: CI ≈ 1, Cro ≈ 2 → estado mixto.
Abajo a la derecha: CI ≈ 5, Cro ≈ 0.2 → estado tipo lisogenia.

¿Por qué ocurre esto?

Las tasas de producción son moderadas y las de degradación relativamente altas.
La represión mutua no es suficientemente fuerte para expulsar el punto intermedio.
El sistema crea tres cuencas de atracción: dependiendo de las condiciones iniciales, puede terminar en cualquiera de los tres atractores.

Tu simulación empezó en CI = 0.1, Cro = 0.1, que está en la cuenca del atractor de lisogenia (abajo a la derecha). Por eso, aunque haya otros atractores, el sistema se dirige hacia ese.


#### caso interesante donde el sistema **no se comporta como un interruptor clásico**
sino que converge a un punto donde CI ≈ Cro (ambos alrededor de 2.1). Vamos a desglosarlo:

***

### **1. Qué muestra el gráfico**

*   Las nulclinas (azul y roja) se cruzan en un punto visible en el centro del plano (CI ≈ 2, Cro ≈ 2).
*   Las flechas del campo de fases apuntan hacia ese cruce desde todas las direcciones → **ese punto es un atractor estable**.
*   No hay divergencia hacia extremos (CI alto/Cro bajo o viceversa), lo que indica que **no hay bistabilidad** con estos parámetros.

***

### **2. Por qué ocurre esto**

*   Tus parámetros:  
    β\_CI = 2.0, β\_Cro = 2.0, α\_CI = 0.20, α\_Cro = 0.20, β₀\_CI = 0.05, β₀\_Cro = 0.05.
*   La expresión basal (β₀) evita que cualquiera de las proteínas llegue a cero.
*   Las tasas de producción y degradación son equilibradas y simétricas.
*   La represión mutua no es suficientemente fuerte para crear dos pozos de atracción separados.
*   Resultado: el sistema se estabiliza en un **punto intermedio donde CI y Cro coexisten**.

***

### **3. Por qué el código dice “lisogenia”**

La función `classify_state()` que usamos compara CI y Cro con un umbral (por defecto ratio\_thresh = 1.0):

```python
state = "lisogenia" if CI_final / Cro_final >= ratio_thresh else "lisis"
```

Como CI\_final ≈ Cro\_final, el ratio ≈ 1.0, y el código interpreta que es lisogenia (porque usamos `>=`).  
**Pero biológicamente esto no es lisogenia ni lisis, sino un estado mixto estable.**  
Esto es una limitación del criterio simplificado que usamos para clasificar.

***

### **4. Cómo se vería la bistabilidad**

*   Si aumentas β\_CI y β\_Cro (por ejemplo, a 3–4) y reduces α\_CI y α\_Cro (a 0.1–0.15), el sistema tendrá dos atractores extremos y un punto inestable en el centro.
*   Las flechas divergirán del centro y convergerán hacia los extremos (CI alto/Cro bajo o viceversa).

***

### **5. Cómo documentarlo para los estudiantes**

Este caso es perfecto para mostrar que:

*   La bistabilidad no es automática: depende de parámetros.
*   Con ciertos valores, el sistema puede ser **monostable** (un único atractor intermedio).
*   La clasificación automática puede inducir a error si no se analiza el campo de fases.

***

¿Quieres que ahora redacte **una sección explicativa para el cuaderno** que muestre:

*   Cómo leer el campo de fases.
*   Qué significa la bistabilidad clásica.
*   Qué ocurre en este caso de monostabilidad y por qué el código lo etiqueta como lisogenia?

¿O prefieres que prepare directamente **Ejercicio 2 interactivo** para explorar condiciones iniciales y ver cómo cambia el destino según parámetros?


#### Respuestas orientativas que los estudiantes podrían desarrollar en la **parte 3 (Analiza)** y en las **preguntas para reflexionar**

***

### **Parte 3: Analiza**

**Pregunta:** ¿Qué ocurre si aumentas la tasa de producción de CI (β\_CI)?  
**Posible respuesta:**  
Al aumentar β\_CI, la nulclina azul se desplaza hacia abajo, lo que favorece que CI alcance valores más altos. Esto amplía la cuenca de atracción de lisogenia y hace que el sistema tienda más fácilmente hacia el atractor con CI alto y Cro bajo.

***

**Pregunta:** ¿Cómo cambia la posición de los atractores al modificar α\_Cro (degradación de Cro)?  
**Posible respuesta:**  
Si α\_Cro aumenta, Cro se degrada más rápido, reduciendo su concentración máxima. Esto desplaza la nulclina roja hacia abajo y puede eliminar el atractor de lisis, dejando el sistema monostable o favoreciendo lisogenia.

***

**Pregunta:** ¿Por qué el sistema se estabiliza en uno de dos estados y no en un punto intermedio (en la bistabilidad clásica)?  
**Posible respuesta:**  
La represión mutua y la cooperatividad generan retroalimentación positiva indirecta: cuando CI gana ventaja, reprime Cro aún más, reforzando su dominio. Esto crea dos pozos de atracción separados y un punto inestable en el centro.

***

### **Preguntas para reflexionar**

**¿Qué parámetros favorecen la lisogenia? ¿Cuáles favorecen la lisis?**

*   Lisogenia: β\_CI alto, α\_CI bajo, β₀\_CI > 0, α\_Cro alto.
*   Lisis: β\_Cro alto, α\_Cro bajo, β₀\_Cro > 0, α\_CI alto.

***

**¿Por qué la bistabilidad es importante para la toma de decisiones biológicas?**  
Porque permite que la célula elija entre dos programas incompatibles (lisis o lisogenia) y mantenga la decisión de forma robusta frente a fluctuaciones y ruido.

***

**¿Qué ocurre si las tasas de producción son bajas y las de degradación altas?**  
El sistema pierde bistabilidad y se vuelve monostable, estabilizándose en un punto intermedio donde CI y Cro coexisten. Esto muestra que la arquitectura del circuito por sí sola no garantiza bistabilidad: depende de parámetros.

***



### Ejercicio 2.  Sensibilidad a las condiciones iniciales y estructura de atractores**

**Objetivo:**  
Comprender cómo las condiciones iniciales determinan el destino del sistema en el lambda switch. Verás que, con ciertos parámetros, el sistema tiene **dos atractores estables** (lisogenia y lisis) y un **punto fijo inestable** en el centro que actúa como frontera entre las cuencas de atracción.

***

### **Parámetros del modelo**

*   **β\_CI, β\_Cro:** tasas máximas de producción de CI y Cro. Valores altos favorecen la acumulación rápida.
*   **α\_CI, α\_Cro:** tasas de degradación. Valores bajos permiten que las proteínas se mantengan.
*   **β₀\_CI, β₀\_Cro:** expresión basal. Si es alta, el sistema tiende a perder bistabilidad porque ambos genes se expresan siempre.
*   **K\_CI, K\_Cro:** constantes de semisaturación. Controlan la afinidad del represor por su operador.
*   **n\_CI, n\_Cro:** coeficientes de Hill. Controlan la cooperatividad. Valores altos (n ≥ 2) hacen que la represión sea más abrupta y favorecen la bistabilidad.

***

### **Valores ejemplo para biestabilidad**

Para ver dos atractores y un punto inestable, usa:

*   β\_CI = 3.0
*   β\_Cro = 3.0
*   α\_CI = 0.15
*   α\_Cro = 0.15
*   β₀\_CI = 0.05
*   β₀\_Cro = 0.05
*   K\_CI = 1.0
*   K\_Cro = 1.0
*   n\_CI = 2
*   n\_Cro = 2

Con estos valores:

*   **Atractor 1 (lisogenia):** CI alto, Cro bajo.
*   **Atractor 2 (lisis):** Cro alto, CI bajo.
*   **Punto inestable:** CI ≈ Cro en el centro (frontera entre cuencas).

***

### **Instrucciones**

1.  Ajusta las condiciones iniciales (CI₀ y Cro₀) con los sliders.
2.  Observa la trayectoria en el campo de fases:
    *   Si CI₀ > Cro₀, el sistema tiende a lisogenia.
    *   Si Cro₀ > CI₀, el sistema tiende a lisis.
    *   Si CI₀ y Cro₀ están cerca del punto inestable, pequeñas variaciones deciden el destino.
3.  Cambia parámetros para ver cómo desaparece la bistabilidad (por ejemplo, aumentando β₀ o reduciendo n).

***

### **Preguntas para reflexionar**

*   ¿Por qué la cooperatividad (n) es clave para la biestabilidad?
*   ¿Qué ocurre si aumentas la expresión basal (β₀)?
*   ¿Cómo se relaciona la frontera entre cuencas con la robustez del sistema frente a ruido?
*   ¿Por qué la decisión depende tanto de las condiciones iniciales?
*   ¿Qué parámetros hacen que la frontera entre cuencas sea más nítida?
*   ¿Por qué este comportamiento es útil para la biología del fago landa?
*   ¿Qué implicaciones tendría si el sistema fuera monoestable en lugar de biestable?


In [ ]:

# --- Nueva versión: dibuja campo de fases y nulclinas en un eje dado ---
def draw_phase_portrait_on_ax(ax, model_func, params, bounds=(0, 6, 0, 6), grid=25):
    CI_min, CI_max, Cro_min, Cro_max = bounds
    ci = np.linspace(CI_min, CI_max, grid)
    cro = np.linspace(Cro_min, Cro_max, grid)
    CI_mesh, Cro_mesh = np.meshgrid(ci, cro)

    # Campo de fases (vector field)
    dCI = np.zeros_like(CI_mesh)
    dCro = np.zeros_like(Cro_mesh)
    for i in range(grid):
        for j in range(grid):
            dydt = model_func(0.0, [CI_mesh[i, j], Cro_mesh[i, j]], params)
            dCI[i, j], dCro[i, j] = dydt

    # Nulclinas (usamos la función existente)
    CI_nc, Cro_nc = compute_nullclines(params, ci_range=ci, cro_range=cro)

    # Dibujar
    ax.quiver(CI_mesh, Cro_mesh, dCI, dCro, color="#7f7f7f",
              angles='xy', scale_units='xy', scale=1.5, alpha=0.6, width=0.003)
    ax.plot(CI_nc, cro, color=PALETTE["nullcline_CI"],  lw=2, label="dCI/dt = 0")
    ax.plot(ci, Cro_nc, color=PALETTE["nullcline_Cro"], lw=2, label="dCro/dt = 0")
    ax.set_xlabel("CI")
    ax.set_ylabel("Cro")
    ax.set_xlim(CI_min, CI_max)
    ax.set_ylim(Cro_min, Cro_max)
    ax.set_title("Campo de fases y nulclinas (interruptor lambda)")
    ax.legend(loc="best")



# =========================
# Ejercicio 2 (mejorado): CI₀, Cro₀ + parámetros
# =========================
import ipywidgets as widgets
from IPython.display import display

# Sliders de condiciones iniciales
CI0_slider  = widgets.FloatSlider(value=0.1, min=0.0, max=10.0, step=0.1, description='CI₀')
Cro0_slider = widgets.FloatSlider(value=0.1, min=0.0, max=10.0, step=0.1, description='Cro₀')

# Sliders de parámetros principales
beta_CI_slider   = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='β_CI')
beta_Cro_slider  = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='β_Cro')
alpha_CI_slider  = widgets.FloatSlider(value=0.2, min=0.05, max=0.5, step=0.01, description='α_CI')
alpha_Cro_slider = widgets.FloatSlider(value=0.2, min=0.05, max=0.5, step=0.01, description='α_Cro')
beta0_CI_slider  = widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.01, description='β₀_CI')
beta0_Cro_slider = widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.01, description='β₀_Cro')

# (Opcional) sliders de K y n si quieres explorar cooperatividad/afinidad
K_CI_slider  = widgets.FloatSlider(value=1.0, min=0.2, max=3.0, step=0.1, description='K_CI')
K_Cro_slider = widgets.FloatSlider(value=1.0, min=0.2, max=3.0, step=0.1, description='K_Cro')
n_CI_slider  = widgets.IntSlider(value=2, min=1, max=4, step=1, description='n_CI')
n_Cro_slider = widgets.IntSlider(value=2, min=1, max=4, step=1, description='n_Cro')

def simulate_phase_and_trajectory(CI0, Cro0,
                                  beta_CI, beta_Cro, alpha_CI, alpha_Cro, beta0_CI, beta0_Cro,
                                  K_CI, K_Cro, n_CI, n_Cro):
    # Config base
    params, t_span, _ = build_config(input_type="none", t_span=(0, 200))
    # Actualizar parámetros
    params["CI"]["beta"]   = beta_CI
    params["Cro"]["beta"]  = beta_Cro
    params["CI"]["alpha"]  = alpha_CI
    params["Cro"]["alpha"] = alpha_Cro
    params["CI"]["beta0"]  = beta0_CI
    params["Cro"]["beta0"] = beta0_Cro
    params["CI"]["K"]      = K_CI
    params["Cro"]["K"]     = K_Cro
    params["CI"]["n"]      = n_CI
    params["Cro"]["n"]     = n_Cro

    y0 = [CI0, Cro0]

    # Simulación
    t, y = run_simulation(model_lambda_switch, params, t_span, y0)

    # Dibujo conjunto: campo + nulclinas + trayectoria
    fig, ax = plt.subplots(figsize=(7, 6))
    draw_phase_portrait_on_ax(ax, model_lambda_switch, params, bounds=(0, 20, 0, 20), grid=25)
    ax.plot(y[0], y[1], color="black", lw=2, label="Trayectoria")
    ax.scatter(y[0][0], y[1][0], color="green", s=50, label="Inicio")
    ax.scatter(y[0][-1], y[1][-1], color="purple", s=50, label="Final")
    ax.legend(loc="best")
    plt.show()

    # Estado final y reporte
    state, CI_final, Cro_final = compute_outcome(t, y)
    print(f"Estado final: {state}")
    print(f"CI final ≈ {CI_final:.2f}, Cro final ≈ {Cro_final:.2f}")

widgets.interactive(
    simulate_phase_and_trajectory,
    CI0=CI0_slider, Cro0=Cro0_slider,
    beta_CI=beta_CI_slider, beta_Cro=beta_Cro_slider,
    alpha_CI=alpha_CI_slider, alpha_Cro=alpha_Cro_slider,
    beta0_CI=beta0_CI_slider, beta0_Cro=beta0_Cro_slider,
    K_CI=K_CI_slider, K_Cro=K_Cro_slider,
    n_CI=n_CI_slider, n_Cro=n_Cro_slider
)



#### respuestas tipo que los estudiantes podrían dar para cada pregunta, redactadas de forma clara y razonada:

***

### **¿Por qué la cooperatividad (n) es clave para la bistabilidad?**

Porque la cooperatividad hace que la represión sea más abrupta: cuando n es alto, pequeñas diferencias en la concentración de CI o Cro producen cambios grandes en la tasa de síntesis del otro. Esto genera retroalimentación positiva indirecta y permite que el sistema se “decante” hacia uno de dos estados estables. Si n = 1 (sin cooperatividad), las curvas son suaves y el sistema tiende a ser monostable.

***

### **¿Qué ocurre si aumentas la expresión basal (β₀)?**

La expresión basal reduce la fuerza de la represión mutua porque ambos genes se siguen expresando aunque el otro esté presente. Si β₀ es alta, el sistema pierde bistabilidad y se estabiliza en un punto intermedio donde CI y Cro coexisten. En casos extremos, puede volverse monostable.

***

### **¿Cómo se relaciona la frontera entre cuencas con la robustez del sistema frente a ruido?**

Una frontera nítida significa que pequeñas fluctuaciones no cambian el destino del sistema: si estás en la cuenca de lisogenia, el ruido no te lleva a lisis. Si la frontera es difusa, el sistema es más sensible al ruido y puede cambiar de estado por variaciones pequeñas.

***

### **¿Por qué la decisión depende tanto de las condiciones iniciales?**

Porque el sistema tiene varias cuencas de atracción separadas por un punto inestable. Las condiciones iniciales determinan en qué cuenca empieza el sistema. Si CI₀ > Cro₀, el sistema se inclina hacia lisogenia; si Cro₀ > CI₀, hacia lisis. Cerca del punto inestable, pequeñas diferencias deciden el destino.

***

### **¿Qué parámetros hacen que la frontera entre cuencas sea más nítida?**

*   β altos y α bajos → atractores más profundos.
*   n altos (cooperatividad) → transición más abrupta.
*   β₀ bajos → represión mutua más efectiva.
    Estos parámetros refuerzan la bistabilidad y reducen la zona de incertidumbre.

***

### **¿Por qué este comportamiento es útil para la biología del fago lambda?**

Porque el fago necesita tomar una decisión clara y mantenerla: lisogenia o lisis son programas incompatibles. La bistabilidad asegura que, una vez elegida una vía, el sistema no cambie por ruido, garantizando la robustez del ciclo vital.

***

### **¿Qué implicaciones tendría si el sistema fuera monostable en lugar de bistable?**

El fago no podría elegir entre dos estilos de vida: siempre acabaría en el mismo estado (por ejemplo, siempre lisogenia). Esto eliminaría la flexibilidad adaptativa y reduciría la supervivencia en entornos cambiantes.


### Ejercicio 3. Papel de la cooperatividad en la biestabilidad**

**Objetivo:**  
Explorar cómo el coeficiente de Hill (n) afecta la forma de las curvas de represión y, por tanto, la estructura de atractores del interruptor lambda. La cooperatividad es clave para que el sistema se comporte como un interruptor: sin ella, el sistema tiende a ser monoestable; con ella, aparece la biestabilidad.

***

### **Concepto clave**

*   **n (coeficiente de Hill):**  
    Controla la cooperatividad en la unión del represor al operador.
    *   Si n = 1 → represión suave → el sistema converge a un punto intermedio (monoestable).
    *   Si n ≥ 2 → represión abrupta → el sistema presenta dos atractores extremos y un punto inestable en el centro (biestable).

***

### **Valores ejemplo para observar el efecto**

*   β\_CI = 3.0, β\_Cro = 3.0
*   α\_CI = 0.15, α\_Cro = 0.15
*   β₀\_CI = 0.05, β₀\_Cro = 0.05
*   K\_CI = 1.0, K\_Cro = 1.0
*   **n variable:** entre 1 y 4.

***

### **Instrucciones**

1.  Ajusta el coeficiente de Hill con el slider.
2.  Observa cómo cambian las nulclinas en el campo de fases y si el sistema pasa de monoestable (un atractor) a biestable (dos atractores y un punto inestable).
3.  Analiza:
    *   ¿Qué ocurre con la frontera entre cuencas cuando n aumenta?
    *   ¿Por qué la cooperatividad es esencial para la toma de decisiones biológicas?

***

### **Preguntas para reflexionar**

*   ¿Por qué el sistema necesita cooperatividad para comportarse como un interruptor?
*   ¿Qué ocurre si n es muy alto? ¿Es más robusto frente a ruido?
*   ¿Cómo se relaciona la forma de las curvas con la existencia de múltiples atractores?

***


In [ ]:
# =========================
# Ejercicio 3: Papel de la cooperatividad (Interactivo)
# =========================
import ipywidgets as widgets
from IPython.display import display

# Slider para coeficiente de Hill
n_slider = widgets.IntSlider(value=1, min=1, max=4, step=1, description='n (cooperatividad)')

def simulate_cooperativity(n):
    # Configuración base
    params, t_span, y0 = build_config(input_type="none", t_span=(0, 200))

    # Ajustar parámetros para favorecer bistabilidad
    params["CI"]["beta"] = 3.0
    params["Cro"]["beta"] = 3.0
    params["CI"]["alpha"] = 0.15
    params["Cro"]["alpha"] = 0.15
    params["CI"]["beta0"] = 0.05
    params["Cro"]["beta0"] = 0.05
    params["CI"]["K"] = 1.0
    params["Cro"]["K"] = 1.0
    params["CI"]["n"] = n
    params["Cro"]["n"] = n

    # Simulación desde condiciones iniciales neutras
    y0 = [10.0, 14.0]
    t, y = run_simulation(model_lambda_switch, params, t_span, y0)

    # Visualización: campo de fases y trayectoria
    fig, ax = plt.subplots(figsize=(7, 6))
    draw_phase_portrait_on_ax(ax, model_lambda_switch, params, bounds=(0, 20, 0, 20), grid=25)
    ax.plot(y[0], y[1], color="black", lw=2, label="Trayectoria")
    ax.scatter(y[0][0], y[1][0], color="green", s=50, label="Inicio")
    ax.scatter(y[0][-1], y[1][-1], color="purple", s=50, label="Final")
    ax.legend(loc="best")
    plt.show()

    # Estado final
    state, CI_final, Cro_final = compute_outcome(t, y)
    print(f"Coeficiente de Hill (n): {n}")
    print(f"Estado final: {state}")
    print(f"CI final ≈ {CI_final:.2f}, Cro final ≈ {Cro_final:.2f}")

# Interfaz interactiva
widgets.interactive(simulate_cooperativity, n=n_slider)

#### REspuestas tipo a las preguntas


##### **¿Por qué el sistema necesita cooperatividad para comportarse como un interruptor?**

Porque la cooperatividad (n > 1) hace que la represión sea más abrupta: cuando una proteína empieza a acumularse, su efecto represor sobre el otro gen se intensifica rápidamente. Esto crea una transición brusca entre estados y permite que el sistema tenga dos atractores bien definidos. Sin cooperatividad (n = 1), la represión es gradual y el sistema tiende a estabilizarse en un punto intermedio (monostable).

***

##### **¿Qué ocurre si n es muy alto (n = 4)? ¿Es más robusto frente a ruido?**

Sí, con n alto las curvas de represión son muy pronunciadas, lo que hace que la frontera entre cuencas sea más nítida. Esto significa que pequeñas fluctuaciones no cambian el destino del sistema, aumentando la robustez frente a ruido. Sin embargo, si n es excesivamente alto, el sistema puede volverse demasiado rígido y menos sensible a señales externas.

***

##### **¿Cómo se relaciona la forma de las curvas con la existencia de múltiples atractores?**

Las nulclinas se vuelven más sigmoides cuando n aumenta, lo que permite que se crucen en tres puntos: dos estables (atractores) y uno inestable. Con n = 1, las curvas son casi lineales y solo se cruzan en un punto estable (monostable).

***
